In [56]:
from scipy.special import comb
import numpy as np
import math
from scipy.stats import nbinom

def truncNegBin_PDF(y, n, p):
    one_f_zero = float(1 - nbinom.pmf(0, n, p))
    return nbinom.pmf(y, n, p) / one_f_zero


def truncNegBin_CDF(a,n,p):
    p=float(p)
    if a <= 0:
        print('Error: y has to be greater than zero')
    else:
        cdf = 0
        for y in range(1,a+1):
            cdf = cdf + truncNegBin_PDF(y,n,p)
        return cdf


def truncNegBin_quantile(q, n, p):
    if q < 0 or q > 1:
        print('Error: quantile has to fit in the intervall [0,1]')
    elif q == 0:
        return -1
    elif q == 1:
        return float('inf')
    else:
        y = 1
        while truncNegBin_CDF(y, n, p) < q:
            y += 1
        return y
    
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies

In [111]:
q = np.array([0])
len(q)

1

In [222]:
""" def truncNegBin_CDF(y, n, p):
    f_zero = float(nbinom.pmf(0, n, p))
    if y > 0:
        return (1/1 - f_zero) * (nbinom.cdf(y, n, p) - f_zero)
    elif y == 0:
        return 0
    else:
        return 'y has to be > 0' """

def truncNegBin_PDF(y, n, p):
    one_f_zero = float(1 - nbinom.pmf(0, n, p))
    return nbinom.pmf(y, n, p) / one_f_zero
    
def truncNegBin_CDF(a,n,p):
    p=float(p)
    if a <= 0:
        return 'y has to be greater than zero'
    else:
        pdf_array = np.array([truncNegBin_PDF(yi, n, p) for yi in range(1, a+1)])
        return np.cumsum(pdf_array)[-1]


def truncNegBin_quantile(quantiles, n, p):
    f_0 = nbinom.pmf(0, n, p)

    # if p(y=0) = 0 no truncation is required
    if (1-f_0) == 1:
        return nbinom.ppf(quantiles, n, p).tolist()
    else:
        y = 1
        while truncNegBin_CDF(y,n,p) < 1:
            y+=1

        cdf_values = [truncNegBin_CDF(yi, n, p) for yi in range(1, y+1)]

        # if single quantile is handed over
        if isinstance(quantiles, float):
            quantile_list = list([quantiles])
            return np.argmax(cdf_values >= quantile_list) + 1
        else:
            quantile_list = []
            for quantile in quantiles:
                y = np.argmax(cdf_values >= quantile) + 1 # cdf_values[0] = cdf(y=1), because of that +1
                quantile_list.append(y)

            return quantile_list


mean = 1
var = 8

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

print(nbinom.cdf(1000, n, p))
print(truncNegBin_CDF(100000, n, p))
#float(1 - nbinom.pmf(0, n, p))

1.0
0.9999999999999994


In [227]:
mean = 1
var = 8

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var
a = 100000

""" pdf_array = np.array([truncNegBin_PDF(yi, n, p) for yi in range(1, a+1)])
np.cumsum(pdf_array) """

def find_first_positive(a, n, p):
    y_values = range(1, a+1)
    probabilities = np.array([truncNegBin_PDF(yi, n, p) for yi in y_values])
    first_positive = np.argmin(probabilities <= 0)
    return y_values[first_positive]

find_first_positive(a, n , p)

1

In [241]:
mean = 200
var = 210000

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

print(1-nbinom.pmf(0, n, p))

pdf_array = np.array([truncNegBin_PDF(yi, n, p) for yi in range(1, 15000+1)])

""" one_f_zero = float(1 - nbinom.pmf(0, n, p))
print(one_f_zero)
print((1/one_f_zero)*nbinom.cdf(0.1, n, p)) """
np.cumsum(pdf_array)


0.7345475197036238


array([0.06883473, 0.109775  , 0.1396419 , ..., 0.99999998, 0.99999998,
       0.99999998])

In [209]:
liste = np.array(range(1,10+1))
liste

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:

# BSP für mean=200 und var=300 -> n=400, p=2/3
#truncNegBin_CDF(100,200,2/3)

#truncNegBin_PDF(100,400,0.66)


liste1=[]

for quantile in quantiles:
    liste1.append(truncNegBin_quantile(quantile, 400, 2/3))
    print(str(quantile), end='\r')

liste1

In [12]:
## Alternative definition of the Negative Binomial
# n is the number of successes = theta
# p is the probability of a single success; (1-p) is the probability of a single failure
# y Anzahl der Misserfolge vor dem nten Erfolg
n = 400
p = 1/3
cdf_values = [truncNegBin_CDF(y, n, p) for y in range(1, n+1)]

liste2 = []
#quantile berechnen
for quantile in quantiles:
    y = 1
    while cdf_values[y-1] < quantile:
        y += 1
    liste2.append(y)
    print(str(quantile), end='\r')

cdf_values = np.array(cdf_values)  # Konvertiere die cdf_values in ein Numpy-Array

liste = []
for quantile in quantiles:
    y = np.argmax(cdf_values >= quantile) + 1
    liste2.append(y)
    print(str(quantile), end='\r')

#liste2 = np.array(liste2)  # Konvertiere die liste in ein Numpy-Array

#liste2


IndexError: list index out of range

In [22]:
theta = 400
mu = 200


#truncNegBin_quantile(0.999, 400, 0.1)
truncNegBin_PDF(100,mu,theta)

OverflowError: math range error

In [17]:
0.01**400

0.0

In [53]:
from scipy.stats import nbinom

def truncNegBin_PDF(y,n,p):
    one_f_zero = float(1 - nbinom.pmf(0, n, p))
    return nbinom.pmf(y, n, p)/one_f_zero

n = 400
p = 2/3
truncNegBin_PDF(1200,n,p)

3.3780230182412004e-255

In [47]:
theta = 401
y = 4
#print(gamma(y + theta) / (gamma(theta)))  #* math.factorial(y)
#gammaln
print(math.exp(gammaln(y + theta)) / math.exp(gammaln(theta)))
print(theta**y)

OverflowError: math range error

In [ ]:


def calculate_quantile(p, n, pdf_func):
    # Finde das Quantil q, für das CDF(q) = p gilt
    quantile = nbinom.ppf(p, n, loc=0, p=pdf_func)
    return quantile

# Beispielverwendung
n = 5
p = 0.3
pdf_func = lambda y: truncNegBin_PDF(y, n, p)

# Berechne das 0.25-Quantil
quantile_25 = calculate_quantile(0.25, n, pdf_func)
print("25-Quantil:", quantile_25)

# Berechne das 0.50-Quantil
quantile_50 = calculate_quantile(0.5, n, pdf_func)
print("50-Quantil:", quantile_50)

# Berechne das 0.75-Quantil
quantile_75 = calculate_quantile(0.75, n, pdf_func)
print("75-Quantil:", quantile_75)

